In [ ]:
import pandas as pd

In [ ]:
stats = pd.read_csv('mvp_stats.csv')

In [ ]:
del stats['Unnamed: 0']

In [ ]:
pd.isnull(stats).sum()

In [ ]:
stats = stats.fillna(0)

In [ ]:
stats.columns

In [ ]:
predictors = ['Age', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year', 'W', 'L', 'W/L%', 'GB', 'PS/G', 'PA/G', 'SRS']

In [ ]:
train = stats[stats['Year'] < 2024]

In [ ]:
test = stats[stats['Year'] == 2024]

In [ ]:
from sklearn.linear_model import Ridge

reg = Ridge(alpha=0.1)

In [ ]:
reg.fit(train[predictors], train['Share'])

In [ ]:
predictions = reg.predict(test[predictors])

In [ ]:
predictions = pd.DataFrame(predictions, columns=['Predictions'], index=test.index)

In [ ]:
predictions

In [ ]:
combine = pd.concat([test[['Player', 'Share']], predictions], axis=1)

In [ ]:
combine


In [ ]:
combine.sort_values(by='Share', ascending=False).head(10)

In [ ]:
from sklearn.metrics import mean_squared_error

mean_squared_error(combine['Share'], combine['Predictions'])

In [ ]:
combine['Share'].value_counts()

In [ ]:
combine = combine.sort_values(by='Share', ascending=False)
combine['Rk'] = list(range(1, combine.shape[0] + 1))

In [ ]:
combine.head(10)

In [ ]:
combine = combine.sort_values(by='Predictions', ascending=False)
combine['Predicted_Rk'] = list(range(1, combine.shape[0] + 1))

In [ ]:
combine.head(10)

In [ ]:
combine.sort_values(by='Share', ascending=False).head(10)

In [ ]:
def find_ap(combine):
    actual = combine.sort_values(by='Share', ascending=False).head(5)
    predicted = combine.sort_values(by='Predictions', ascending=False)
    ps = []
    found = 0
    seen = 1
    for i, r in predicted.iterrows():
        if r['Player'] in actual['Player'].values:
            found += 1
            ps.append(found/seen)
        seen += 1
    return sum(ps)/len(ps)

In [ ]:
find_ap(combine)

In [ ]:
years = list(range(1995, 2025))

In [ ]:
aps = []
all_preds = []
for year in years[5:]:
    train = stats[stats['Year'] < year]
    test = stats[stats['Year'] == year]
    reg = Ridge(alpha=0.1)
    reg.fit(train[predictors], train['Share'])
    predictions = reg.predict(test[predictors])
    predictions = pd.DataFrame(predictions, columns=['Predictions'], index=test.index)
    combine = pd.concat([test[['Player', 'Share']], predictions], axis=1)
    all_preds.append(combine)
    aps.append(find_ap(combine))

In [ ]:
sum(aps)/len(aps)

In [ ]:
def add_ranks(combine):
    combine = combine.sort_values(by='Share', ascending=False)
    combine['Rk'] = list(range(1, combine.shape[0] + 1))
    combine = combine.sort_values(by='Predictions', ascending=False)
    combine['Predicted_Rk'] = list(range(1, combine.shape[0] + 1))
    combine['Difference'] = combine['Rk'] - combine['Predicted_Rk']
    return combine

In [ ]:
ranking = add_ranks(all_preds[1])
ranking[ranking['Rk'] < 6].sort_values(by='Difference', ascending=False)

In [ ]:
def backtest(stats, model, year, predictors):
    aps = []
    all_preds = []
    for year in years[5:]:
        train = stats[stats['Year'] < year]
        test = stats[stats['Year'] == year]
        model.fit(train[predictors], train['Share'])
        predictions = model.predict(test[predictors])
        predictions = pd.DataFrame(predictions, columns=['Predictions'], index=test.index)
        combine = pd.concat([test[['Player', 'Share']], predictions], axis=1)
        combine = add_ranks(combine)
        all_preds.append(combine)
        aps.append(find_ap(combine))
    return sum(aps)/len(aps), aps, pd.concat(all_preds)

In [ ]:
mean_ap, aps, all_preds = backtest(stats,reg, years[5:], predictors)

In [ ]:
mean_ap

In [ ]:
all_preds[all_preds['Rk']<=5].sort_values(by='Difference', ascending=False)

In [ ]:
pd.concat([pd.Series(reg.coef_), pd.Series(predictors)], axis=1).sort_values(by=0, ascending=False)

In [ ]:
ratios = stats[['PTS', 'AST', 'STL', 'BLK', '3P', 'Year']].apply(lambda x: x/x.mean())

In [ ]:
ratios

In [ ]:
stats[['PTS_T', 'AST_R', 'STL_R', 'BLK_R', '3P_R']] = ratios[['PTS', 'AST', 'STL', 'BLK', '3P']]

In [ ]:
stats.head()

In [ ]:
predictors+=['PTS_T', 'AST_R', 'STL_R', 'BLK_R', '3P_R']

In [ ]:
mean_ap, aps, all_preds = backtest(stats, reg, years[5:], predictors)

In [ ]:
mean_ap

In [ ]:
stats['NPos'] = stats['Pos'].astype('category').cat.codes

In [ ]:
stats['NTm'] = stats['Tm'].astype('category').cat.codes

In [ ]:
stats['NTm'].value_counts()    

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=200, random_state=1, min_samples_split=10)

mean_ap, aps, all_preds = backtest(stats, rf, years[5:], predictors)

In [ ]:
mean_ap

In [ ]:
mean_ap, aps, all_preds = backtest(stats, reg, years[5:], predictors)

In [ ]:
mean_ap